<a href="https://colab.research.google.com/github/Alessa2005/Practicas-Programacion/blob/main/Practica2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as ptl
import numpy as np


class Cola:
    """Clase Cola para manejar las colas de clientes.
    Esta clase permite agregar y quitar clientes de una cola, así como verificar si está vacía.
    """
    def __init__(self):
        self.cola_clientes = []

    def cola_vacia(self):
        """Verifica si la cola está vacía."""
        return len(self.cola_clientes) == 0

    def agregar_en_cola(self, cliente):
        """Agrega un cliente a la cola."""
        return self.cola_clientes.append(cliente)

    def quitar_en_cola(self):
        """Elimina y devuelve el primer cliente de la cola, si no está vacía."""
        if not self.cola_vacia():
            return self.cola_clientes.pop(0)


class Cliente:
    """Clase Cliente para manejar los datos de cada cliente.
    Cada cliente tiene hora de llegada, hora de atención, hora de salida y tipo (Premiere o Regular).
    """
    def __init__(self, hora_llegada=None, hora_atencion=None, hora_salida=None, tipo=False):
        self.hora_llegada = hora_llegada
        self.hora_atencion = hora_atencion
        self.hora_salida = hora_salida
        self.tipo_cliente = tipo


class CentroLlamadas:
    """Clase CentroLlamadas que simula un centro de llamadas con clientes y servidores.
    Maneja la cola de clientes y la asignación de servidores.
    """
    def __init__(self, numero_clientes=550, numero_servidores=43,
                 minimo_atencion=1, maximo_atencion=81, minimo_inter=1,
                 maximo_inter=3):
        # Inicializa las colas y los parámetros de la simulación.
        self.atender = Cola()
        self.cola_premiere = Cola()
        self.cola_regular = Cola()
        self.numero_clientes = numero_clientes
        self.numero_servidores = numero_servidores
        self.minimo_inter = minimo_inter
        self.maximo_inter = maximo_inter
        self.minimo_atencion = minimo_atencion
        self.maximo_atencion = maximo_atencion
        self.servidores = [None] * numero_servidores
        self.reloj = 0
        self.tiempo_espera_premiere = []
        self.tiempo_espera_regular = []

    def generar_interarribos(self):
        """Genera un tiempo aleatorio entre llegadas de clientes."""
        return np.random.uniform(self.minimo_inter, self.maximo_inter)

    def generar_tiempo_atencion(self):
        """Genera un tiempo aleatorio de atención para los clientes."""
        return np.random.uniform(self.minimo_atencion, self.maximo_atencion)

    def generar_tipo_cliente(self):
        """Genera aleatoriamente si un cliente es de tipo Premiere (1 de cada 7 es Premiere)."""
        return int(np.random.uniform(1, 7)) == 1

    def servidor_libre(self):
        """Verifica si hay algún servidor libre. Si lo hay, lo devuelve."""
        for i in range(self.numero_servidores):
            if self.servidores[i] is None:
                return i
            elif self.reloj >= self.servidores[i].hora_salida:
                self.servidores[i] = None
                return i
        return None

    def asignar_cliente(self, cliente):
        """Asigna un cliente a un servidor libre si lo hay, o lo pone en cola si no."""
        s_libre = self.servidor_libre()
        if s_libre is not None:
            self.servidores[s_libre] = cliente
            tiempo_atencion = self.generar_tiempo_atencion()
            cliente.hora_atencion = self.reloj
            cliente.hora_salida = self.reloj + tiempo_atencion
            tiempo_espera = cliente.hora_atencion - cliente.hora_llegada
            if cliente.tipo_cliente:
                self.tiempo_espera_premiere.append(tiempo_espera)
            else:
                self.tiempo_espera_regular.append(tiempo_espera)

            print("Cliente asignado al servidor: " + str(s_libre)
                  + "\nTiempo de atención: " + str(tiempo_atencion)
                  + "\nSu hora de salida fue: " + str(cliente.hora_salida))

        else:
            self.atender.agregar_en_cola(cliente)

    def revisar_cola(self):
        """Asigna clientes de la cola a servidores si hay servidores libres."""
        while not self.atender.cola_vacia() and self.servidor_libre() is not None:
            cliente = self.atender.quitar_en_cola()
            print("Hora de llegada: " + str(cliente.hora_llegada) + " mins")
            self.asignar_cliente(cliente)

    def simulacion(self):
        """Realiza la simulación del centro de llamadas."""
        for i in range(self.numero_clientes):
            tipo_cliente = self.generar_tipo_cliente()
            cliente = Cliente(hora_llegada=self.reloj, tipo=tipo_cliente)
            if tipo_cliente:
                self.cola_premiere.agregar_en_cola(cliente)
                print("Cliente PREMIERE asignado. Su hora de llegada fue: " + str(self.reloj))
            else:
                self.cola_regular.agregar_en_cola(cliente)
                print("Cliente REGULAR asignado. Su hora de llegada fue: " + str(self.reloj))

            self.asignar_cliente(cliente)
            self.revisar_cola()
            self.reloj += 1

        # Cálculo de estadísticas finales.
        if self.tiempo_espera_premiere:
            promedio_premiere = np.mean(self.tiempo_espera_premiere)
            print("Promedio de espera para clientes PREMIERE: " + str(promedio_premiere))
        else:
            print("No hay clientes PREMIERE atendidos.")

        if self.tiempo_espera_regular:
            promedio_regular = np.mean(self.tiempo_espera_regular)
            print("Promedio de espera para clientes REGULAR: " + str(promedio_regular))
        else:
            print("No hay clientes REGULAR atendidos.")


class Estadisticas:
    """Clase Estadisticas para generar gráficos y cálculos de promedio de espera."""
    def grafica(self):
        """Genera un gráfico de número de clientes vs tiempo promedio de espera."""
        n_clientes = [100, 200, 320, 550, 1500, 3500, 4000, 10000]
        promedios = [19, 19, 35, 43, 43, 43, 43, 43]
        ptl.plot(n_clientes, promedios)
        return ptl.show()

    def promedio_espera(self):
        """Calcula el promedio de espera de clientes PREMIERE a partir de datos dados."""
        esperas_premiere = [0, 4, 11, 0.51, 2.58, 1.97, 2.44, 2.5]
        promedios_esperas = np.mean(esperas_premiere)
        print("El promedio de esperas es de: ", promedios_esperas)


if __name__ == "__main__":
    # center = CentroLlamadas()
    # center.simulacion()
    estadisticas = Estadisticas()
    estadisticas.grafica()
    estadisticas.promedio_espera()

print("1) ¿Cuál es el mínimo número de líneas que se deben mantener abiertas "
      "para darle servicio a los clientes sin que se sature el sistema?")
print("R: Podemos notar que a las 43 líneas ya tenemos una función acotada, "
      "es decir, ya podemos decir que a ese número nuestras líneas ya no se saturan.")

print("2) ¿Cuál es el estimado de tiempo máximo de atención de los clientes premiere?")
print("R: El promedio de espera de los clientes premiere es de 3.125 minutos.")

print("Integrantes: Robles Gómez Alessandra, Evangelista Cortés Carlos Adrián, "
      "Solís Rodríguez José Ramón.")
